In [16]:
import os
import glob
import pandas as pd 
from chromolooper import sgls
import re
from myvariant import MyVariantInfo
import numpy as np
import subprocess as sp
from chromolooper import sgls
from time import sleep


pd.options.mode.chained_assignment = None  # default='warn'

os.chdir('<project-dir>')

outdir = 'results/hg38/eqtl/eqtl_catalogue/'
os.makedirs(outdir, exist_ok=True)

In [5]:
old_fn = 'results/hg38/refs/gencode/v30/gencode.v30.annotation.w_genetypes.bed'
fn = 'results/hg38/refs/gencode/v30/gencode.v30.annotation.genes_only.bed.gz'
gencode_df = pd.read_table(fn, header=None)
gencode_df = gencode_df.iloc[:, [0,1,2,5,6,7,8]]
gencode_df.columns =['chrom', 'start', 'end', 'strand', 'genename', 'gene_id', 'genetype']
# gencode_df = gencode_df.loc[gencode_df.type == 'gene'].drop('type', axis=1)
gencode_df.loc[:,'gene_id'] = gencode_df.loc[:,'gene_id'].str.replace('\.[0-9]*', '', regex=True)

<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:7: SyntaxWarning: invalid escape sequence '\.'
/mnt/BioHome/jreyna/tmp/ipykernel_1826854/4159143216.py:7: SyntaxWarning: invalid escape sequence '\.'
  gencode_df.loc[:,'gene_id'] = gencode_df.loc[:,'gene_id'].str.replace('\.[0-9]*', '', regex=True)


## Load all eQTL data

In [6]:
genome_sizes = 'http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes'

In [17]:
# helper function to find snps only
def is_snp(variant_id):
    var1, var2 = variant_id.split('_')[-2:]
    if len(var1) == 1 and len(var2) == 1:
        return(True)
    return(False)

def get_name(sr):
    
    if type(sr['rsid']) == str:
        res = sr['rsid'] + '-' + sr['genename']
    else:
        res = sr['variant'] + '-' + sr['genename']
    return(res)

unique_snps = set()
credible_set_fns = glob.glob('results/hg38/eqtl/eqtl_catalogue/*.credible_sets.tsv.gz')

credible_set_fns = ['results/hg38/eqtl/eqtl_catalogue/QTD000474.credible_sets.tsv.gz',
                    'results/hg38/eqtl/eqtl_catalogue/QTD000479.credible_sets.tsv.gz',
                    'results/hg38/eqtl/eqtl_catalogue/QTD000489.credible_sets.tsv.gz',
                    'results/hg38/eqtl/eqtl_catalogue/QTD000499.credible_sets.tsv.gz',
                    'results/hg38/eqtl/eqtl_catalogue/QTD000504.credible_sets.tsv.gz',
                    'results/hg38/eqtl/eqtl_catalogue/QTD000509.credible_sets.tsv.gz']
                    
for fn in credible_set_fns:
    tdf = pd.read_table(fn)
    tdf = tdf.loc[tdf.variant.apply(is_snp)]
    tdf = tdf.merge(gencode_df, on='gene_id')
    
    ec_dataset_id = os.path.basename(fn).split('.')[0]

    
    tdf['chrom'] = tdf.variant.apply(lambda x: x.split('_',1)[0])
    tdf['end'] = tdf.variant.apply(lambda x: int(x.split('_',2)[1]))
    tdf['start'] = tdf['end'] - 1
    tdf['name'] = tdf.apply(get_name, axis=1)
    tdf['score'] = 1
    tdf['strand'] = '+'
    tdf['color'] = '117,117,117'
    tdf.sort_values(['chrom', 'end', 'start'], inplace=True)

    # save the final
    rgbpeak_df = tdf[['chrom', 'start', 'end', 'name', 'score', 'strand', 'start', 'end', 'color']]
    peak_fn = os.path.join(outdir, '{}.rgbpeak'.format(ec_dataset_id))
    rgbpeak_df.to_csv(peak_fn, sep='\t', index=False, header=False)
    
    # # compress
    # peak_gz_fn = os.path.join(outdir, '{}.rgbpeak.gz'.format(ec_dataset_id))
    # cmd = 'bgzip -f {}'.format(peak_fn)
    # j = sp.check_output(cmd, shell=True)
    # print(j.decode())

    
    # # tabix
    # cmd = 'tabix -f -p bed {}'.format(peak_gz_fn)
    # j = sp.check_output(cmd, shell=True)
    # print(j.decode())
    
    bigpeak_fn = os.path.join(outdir, '{}.rgbpeak.bb'.format(ec_dataset_id))
    sgls.bed_to_bigbed(peak_fn, bigpeak_fn, genome_sizes, verbose=True)

    # need time between runs of bed_to_bigbed
    sleep(2)

pass1 - making usageList (22 chroms): 46 millis
pass2 - checking and writing primary data (59350 records, 9 fields): 209 millis


pass1 - making usageList (22 chroms): 61 millis
pass2 - checking and writing primary data (63228 records, 9 fields): 219 millis


pass1 - making usageList (22 chroms): 75 millis
pass2 - checking and writing primary data (66236 records, 9 fields): 234 millis


pass1 - making usageList (22 chroms): 53 millis
pass2 - checking and writing primary data (57544 records, 9 fields): 183 millis


pass1 - making usageList (22 chroms): 51 millis
pass2 - checking and writing primary data (65486 records, 9 fields): 216 millis


pass1 - making usageList (22 chroms): 69 millis
pass2 - checking and writing primary data (47375 records, 9 fields): 167 millis
